In [ ]:
import numpy as np
import random as r
from matplotlib import pyplot as plt
import pandas as pd

In [ ]:
def get_Q(I,M,L,lead_time):
  #I is current inventory
  if I < L and 0<= lead_time <= 1:
    QU = M-I
  elif lead_time ==20 and I < L:
    QU = M-I
  else:
    QU = 0
  return QU 

In [ ]:
def order_cost(Q):
  if Q > 0:
   return 60 + 5*Q
  else:
    return 0

In [ ]:
def back_cost(I):
  if I < 0:
    return -I*4
  else:
    return 0 

In [ ]:
def inv_cost(I):
  if I > 0:
    return I
  else:
    return 0

In [ ]:
#funtion to calculate revenue
def selling(inv,monthly_demand,order_reciept,start_inv):
  b = start_inv
  m = inv
  revenue = []
  for i in range(1):
    if order_reciept[i]+b[i] > 0:
      sales = min(order_reciept[i]+b[i],monthly_demand[i])*10
      revenue.append(sales)

  for i in range(1,len(inv),1):
    z = m[i-1] 
    if z > 0 :
      sales = min(z,monthly_demand[i])*5
      if monthly_demand[i] > z:
        sales = sales+ 10*min(b[i]-z,monthly_demand[i]-z)
    elif b[i] > 0:
      if m[i-1] >= 0:
        sales = min(b[i],monthly_demand[i])*10
      else:
        sales = min(order_reciept[i],monthly_demand[i]-m[i-1])*10
    else:
      sales = 0
    revenue.append(sales)
  return revenue

In [ ]:
def simulation(no_of_replication = 1, length_of_each_run = 100 ):
  L = length_of_each_run
  n = no_of_replication
  mean_profit = []
  mean_monthly_cost = []
  sl = [] # cycle service level
  data = pd.DataFrame(data = None)
  for i in range(n):
    z = -1
    order_reciept = int(L+2)*[0] # creating this for assigning order quantity recival to particular month because suppose if lead time is more than 1 month then adding order quantity in second month
    I = 50 #initial inventory
    monthly_cost = []
    monthly_orders = []
    monthly_demand = []
    start_inv = []
    t = 0 #initial month
    L_time = 20
    inv = []
    lead = []
    while t < L: # run for L month
      I = I + order_reciept[t] # assigned order quantity if lead time is more than 1 month 
      start_inv.append(I)
      d_t = 0 # initialize demand time in given month
      dm = [] 
      while d_t < 1: # writing code to evaluate total demand occured in a given month
        d_t = d_t + r.expovariate(15)
        pr = r.uniform(0,1)
        if pr >= 0 and pr<= 0.5:
          D = 1
        elif 0.5 < pr <= 0.75:
          D = 2
        elif 0.75 < pr <= 7/8:  
          D = 3
        else:
          D = 4
        dm.append(D)
      demand = sum(dm) # this will give total demand of that month
      monthly_demand.append(demand)

      I = I -demand

      if t >= 1:
        z = inv[t-1]   - monthly_demand[t]
      if z > 0:
        I = I - z
      Q = get_Q(I,50,30,L_time)  ######
      monthly_cost.append(order_cost(Q)+back_cost(I)+ inv_cost(I))
      inv.append(I)
      monthly_orders.append(Q)
      if Q > 0:
        L_time = r.uniform(0.25,1.25)
      else:
        L_time = 0
      lead.append(L_time)
      if 0.25 <= L_time <= 1 and Q > 0: #if lead time is less than 1 month that means we are getting our ordered quantity in next month
        order_reciept[t+1] = Q
      elif L_time > 1 and Q > 0: # therefore in next month inventory becomes on hand inventory plus recieved quanity
        order_reciept[t+2] = Q
        order_reciept[t+1] = 0
      t = t + 1 
    #print('inv',inv)
    #print('demand',monthly_demand)
    #print('order_reciept',order_reciept)
    #print('leadtime',lead) 
    #print('monthlycost',monthly_cost)
    seling = selling(inv,monthly_demand,order_reciept,start_inv)
    mean_profit.append(-np.mean(monthly_cost)+ np.mean(seling)) 
    mean_monthly_cost.append(np.mean(monthly_cost))
    #data['START INVENTORY'] = start_inv[:10]
    data['MONTHLY DEMAND'] = monthly_demand[:10]
    data['ORDER RECEIPT'] = order_reciept[:10]
    data['END INVENTORY'] = inv[:10]
    data['MONTHLY ORDER'] = monthly_orders[:10]
    data['MONTHLY COST'] = monthly_cost[:10]
    data['MONTHLY SELLING'] = seling[:10]
    modified_inv = [50,*inv]
    #print('mod inv',modified_inv)
    ser_level_list = []
    sl_count = 0
    for i in range(len(monthly_demand)):
      ser_level_list.append(modified_inv[i] + order_reciept[i] - monthly_demand[i])
    for service_level in ser_level_list:
      if service_level >= 0:
        sl_count += 1
    sl.append(sl_count/len(monthly_demand))
  return mean_profit,mean_monthly_cost,sl ,data

$mean\_monthly\_cost$ is a list of length = no. of replications . Each element represent average monthly cost averaged over length of each run. Similarly for $mean\_profit$.

$sl$ is a list of length = no. of replications. Each element shows averaged cycle service level averaged over length of each run.

$data$ gives a datafram containing first 10 month cost, inventory, order_receipt, monthly_order etc. for the last simulation run. It will be usefull for verifying the code.

Now lets run a example simulation with just one run.

In [ ]:
#example run
mmp, mmc ,sl_list ,data= simulation(1)
data

,MONTHLY DEMAND,ORDER RECEIPT,END INVENTORY,MONTHLY ORDER,MONTHLY COST,MONTHLY SELLING
0,38,0,12,38,262,380
1,38,0,-26,0,104,60
2,34,38,-22,72,508,380
3,26,72,24,26,214,480
4,34,26,16,34,246,220
5,37,34,13,37,258,290
6,34,37,16,34,246,275
7,24,34,26,24,206,160
8,33,24,17,33,242,200
9,38,0,-21,0,84,85


##part a

#Now we run simulation for 10 replication each of length 100 months. **NO WARM UP PERIOD SPECIFIED FOR THIS PART OF THE QUESTION

In [ ]:
mean_profit, mean_cost,csl_list ,data = simulation(no_of_replication = 10, length_of_each_run = 100)

In [ ]:
print('mean profit for 10 run', mean_profit)

mean profit for 10 run [14.469999999999999, 13.360000000000014, 19.930000000000007, 16.129999999999995, 3.6399999999999864, 2.1999999999999886, 19.70999999999998, 23.94999999999999, 6.4199999999999875, 7.1200000000000045]


In [ ]:
print(np.mean(mean_profit))

12.692999999999994


In [ ]:
print('mean monthly for 10 run', mean_cost)

mean monthly for 10 run [233.48, 231.89, 242.62, 241.12, 226.61, 219.05, 251.44, 237.0, 234.03, 235.38]


In [ ]:
print(np.mean(mean_cost))

235.26200000000003


In [ ]:
print('cycle service level for 10 run', csl_list)

cycle service level for 10 run [0.65, 0.7, 0.52, 0.62, 0.67, 0.77, 0.51, 0.59, 0.67, 0.7]


##part b

Now lets calculate CI for cost and profit

In [ ]:
import statistics

In [ ]:
mean_mc = np.mean(mean_cost)
print('mean of 90% CI for mean_monthly_cost',mean_mc)

mean of 90% CI for mean_monthly_cost 235.26200000000003


In [ ]:
#90% CI FOR MEAN COST
S = (statistics.variance(mean_cost))**0.5 #standard deviation
R = 10
t = 1.833 # for alpha = 0.1 
hw_cost  = t*S/np.sqrt(R)
print('#90% CI half width for mean_monthly_cost',hw_cost)

#90% CI half width for mean_monthly_cost 5.137374184576784


In [ ]:
mean_mp = np.mean(mean_profit)
print('mean of 90% CI for monthly_profit',mean_mp)

mean of 90% CI for monthly_profit 12.692999999999994


In [ ]:
#90% CI FOR MEAN profit
S = (statistics.variance(mean_profit))**0.5
R = 10
t = 1.833 #alpha = 0.1, 
hw_profit  = t*S/np.sqrt(R)
print('#90% CI half width for monthly_profit',hw_profit)

#90% CI half width for monthly_profit 4.347198704062307


lets calculate no. of replication for half width of monthly cost to be under 5.

In [ ]:
#for hw = 5 of monthly cost
R = 10
R_req = R*((hw_cost)**2)/25
print('#for hw = 5 of monthly cost requred R',np.ceil(R_req))

#for hw = 5 of monthly cost requred R 11.0


Now we run simulation again with required R.

In [ ]:
new_mp, new_mc,new_sl_list ,data = simulation(int(R_req))


In [ ]:
mean_new_mc = np.mean(new_mc)
print(mean_new_mc)

228.59900000000002


In [ ]:
#90% CI FOR  new MEAN profit
S = (statistics.variance(new_mc))**0.5
R = R_req
t = 1.79 #alpha = 0.1, 
hw_new_cost  = t*S/np.sqrt(R)
print('NEW HALF WIDTH FOR 90% CI OF MONTHLY COST',hw_new_cost)

NEW HALF WIDTH FOR 90% CI OF MONTHLY COST 3.7148418401497736


Now half width for 90% CI of monthly cost is around 2.66 which is less than 5 as required.

simulation optimization part is done in 2nd file.